In [10]:
#<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< INSTALLATION GUIDE>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 

'''pip install nltk

pip install spacy==2.3.5

pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz

pip install pyresparser
pip install -U pyresparser pdfplumber unidecode pandas'''


'pip install nltk\n\npip install spacy==2.3.5\n\npip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz\n\npip install pyresparser\npip install -U pyresparser pdfplumber unidecode pandas'

In [11]:
# import pdfplumber
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords 
from string import punctuation
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from heapq import nlargest
from collections import defaultdict
import pandas as pd 
from nltk.collocations import *
import base64,random

In [12]:
import os
import pandas as pd
import base64
from pyresparser import ResumeParser
import pdfplumber
import re
from typing import List, Dict, Optional

# ====================== Cấu hình ======================
RESUMES_DIR = 'cv'
OUTPUT_CSV = 'resume_data.csv'
UNWANTED_PATTERNS = r'[^\w\s,.!?@#\$%&\*\(\)\-\+=\/\'"–—∶−]'  # Giữ lại các ký tự hợp lệ
# =======================================================

def preprocess_text(text: str) -> str:
    """
    Chuẩn hóa text từ PDF:
    - Xóa các ký tự không mong muốn
    - Chuẩn hóa khoảng trắng
    - Xử lý xuống dòng
    """
    # Thay thế nhiều khoảng trắng và xuống dòng
    text = re.sub(r'\s+', ' ', text)
    # Xóa các ký tự đặc biệt không mong muốn
    text = re.sub(UNWANTED_PATTERNS, '', text)
    # Chuẩn hóa dấu gạch ngang
    text = re.sub(r'[\u2013\u2014\u2212]', '-', text)
    return text.strip()

def extract_name_from_text(text: str) -> Optional[str]:
    """
    Trích xuất tên từ text CV sử dụng regex nâng cao
    """
    # Tìm kiếm mẫu tên ở phần đầu CV
    name_pattern = r'''
        ^                   # Bắt đầu chuỗi
        ([A-ZÀ-Ỹ][a-zà-ỹ]+  # Tên đầu viết hoa
        (?:\s+              # Khoảng trắng
        [A-ZÀ-Ỹ][a-zà-ỹ.]*  # Các từ tiếp theo (cho tên đệm, họ)
        ){1,4})             # Tổng 2-4 từ
        (?=\s*(?:[A-Z]|$))  # Lookahead cho ký tự viết hoa hoặc kết thúc
    '''
    match = re.search(name_pattern, text, re.VERBOSE | re.MULTILINE)
    return match.group(1) if match else None

def parse_resume(filepath: str) -> Dict:
    """
    Xử lý file PDF và trích xuất thông tin
    """
    try:
        # Trích xuất text thô
        with pdfplumber.open(filepath) as pdf:
            text = "\n".join([page.extract_text(layout=True) for page in pdf.pages])
            clean_text = preprocess_text(text)
            
        # Phân tích bằng ResumeParser
        parsed_data = ResumeParser(filepath).get_extracted_data()
        
        # Kết hợp dữ liệu
        return {
            'name': extract_name_from_text(clean_text) or parsed_data.get('name'),
            'email': parsed_data.get('email'),
            'mobile_number': parsed_data.get('mobile_number'),
            'skills': parsed_data.get('skills', []),
            'education': parsed_data.get('degree') or parsed_data.get('education', []),
            'experience_years': parsed_data.get('total_experience'),
            'file_path': filepath,
            'raw_text': clean_text
        }
    except Exception as e:
        print(f"Lỗi khi xử lý {filepath}: {str(e)}")
        return None

def main():
    # Khởi tạo DataFrame
    resume_data = []
    
    # Duyệt qua các file PDF
    for filename in os.listdir(RESUMES_DIR):
        if filename.lower().endswith('.pdf'):
            filepath = os.path.join(RESUMES_DIR, filename)
            if data := parse_resume(filepath):
                resume_data.append(data)
    
    # Tạo DataFrame
    df = pd.DataFrame(resume_data)
    
    # Xử lý cột skills
    df['skills'] = df['skills'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
    
    # Sắp xếp cột
    columns_order = [
        'name', 'email', 'mobile_number', 'skills',
        'education', 'experience_years', 'file_path', 'raw_text'
    ]
    df = df[columns_order]
    
    # Lưu kết quả
    df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')
    print(f"Đã xử lý xong {len(df)} CV. Kết quả lưu tại: {OUTPUT_CSV}")
    return df

if __name__ == "__main__":
    merged_df = main()
    print(merged_df)

c:\Users\Admin\anaconda3\envs\talenthive\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.1 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Đã xử lý xong 20 CV. Kết quả lưu tại: resume_data.csv
                         name  \
0               Le Thanh Long   
1               Le Thanh Long   
2                     NHAN HO   
3               thanhduycao |   
4                  NGUYEN THI   
5               Lê Thùy Dương   
6               Dao Khanh Duy   
7           Ho Quoc Thien Anh   
8                   HUYNH THI   
9                 Nguyen Tien   
10                 NGUYEN VAN   
11               Nguyen Hoang   
12  | linkedin.com/in/haiph02   
13                 PHAM QUANG   
14                  Phan Minh   
15           KOREA UNIVERSITY   
16                 Phuoc Phan   
17               PHONG NGUYEN   
18              Le Thanh Long   
19         Nguyễn Huyền Trang   

                                                email mobile_number  \
0                              attentionocr@gmail.com  096-997-3012   
1                              attentionocr@gmail.com  096-997-3012   
2                           hosybaonhan

In [13]:
merged_df.columns

Index(['name', 'email', 'mobile_number', 'skills', 'education',
       'experience_years', 'file_path', 'raw_text'],
      dtype='object')

In [14]:
# df2 = merged_df.copy()

In [15]:
merged_df.to_csv('cv.csv')

In [16]:
df=pd.read_csv("cv.csv")

In [17]:
df

,Unnamed: 0,name,email,mobile_number,skills,education,experience_years,file_path,raw_text
0,0,Le Thanh Long,attentionocr@gmail.com,096-997-3012,"Mock, Research, Communication, Scheduling, Css...",[],0.00,cv\1.0.pdf,Le Thanh Long Research and Development Phone09...
1,1,Le Thanh Long,attentionocr@gmail.com,096-997-3012,"Research, Communication, Scheduling, Css, Flas...",['MLOPS'],0.00,cv\2.0.pdf,Le Thanh Long Research and Development Birth26...
2,2,NHAN HO,hosybaonhan2004@gmail.com,022-2027,"Linux, Research, Communication, Flask, Mobile,...",[],0.00,cv\bao nhan ho sy-cv.pdf,BAO-NHAN HO-SY PYTHON BACK-END DEVELOPER Short...
3,3,thanhduycao |,ctduy20@apcs.fitus.edu.vn,NaN,"Research, Content, Communication, Flask, Syste...",[],1.08,cv\CaoThanhDuy_Resume.pdf,THANH-DUY CAO ctduy20@apcs.fitus.edu.vn thanh...
4,4,NGUYEN THI,chauthi1704@gmail.com,8438728569,"Communication, Css, User experience, Mobile, S...",[],0.00,cv\ChauThi_resume.pdf,"NGUYEN THI CHAU THI 470, Tran Dai Nghia, Ngu H..."
5,5,Lê Thùy Dương,ltduong2911@gmail.com,0707292696,"Website, Lan, Email, Word, Excel",[],0.00,cv\CV Thuy Duong (Việt)1743071666667.pdf,Lê Thùy Dương Nhân viên văn phòng Ngày sinh 29...
6,6,Dao Khanh Duy,khanhhh.duy@gmail.com,022-2026,"Docker, Communication, Css, User experience, E...",['Bachelor of Science in Computer Science'],1.42,cv\DaoKhanhDuy.pdf,Dao Khanh Duy Sept 2004 0789-402-189 khanhhh...
7,7,Ho Quoc Thien Anh,hoquocthienanh@gmail.com,0905500293,"Hotel, Communication, Seaborn, English, Presen...","[""Bachelor's degree Information Technology""]",2.58,cv\HoQuocThienAnh.pdf,"Ho Quoc Thien Anh Vietnam, Da Nang City hoquo..."
8,8,HUYNH THI,huynhthiphuoc04@gmail.com,022-2026,"Teaching, Java, Workflow, Docker, Php, Css, Co...",[],1.25,cv\huynhthiphuoc_resume1.pdf,HUYNH THI PHUOC July 2004 0355-176-782 huynh...
9,9,Nguyen Tien,dungbk.nguyentien@g,0973621049,"Linux, Research, Docker, Unix, Agile, System, ...",['\uf0b1'],0.00,cv\Nguyen-Tien-Dung.pdf,Nguyen Tien Dung 0973621049 Male Developer ...


In [18]:
import os
import pandas as pd
from pyresparser import ResumeParser
import pdfplumber
import re

In [19]:
def pdf_to_base64(file_path):
    #with pdfplumber.open(filepath) as pdf
    with open(file_path, 'rb') as f:
    
        encoded_pdf = base64.b64encode(f.read()).decode('utf-8')

    return encoded_pdf

In [20]:
encoded_files=[]
for i in df["file_path"].values:
    print()
    encoded_files.append(pdf_to_base64(i))

In [21]:
df["pdf_to_base64"]=encoded_files

In [22]:
df.to_csv("updated_cv.csv",index=False)